# Set working dir

In [2]:
import os 
import warnings

warnings.filterwarnings("ignore")
os.chdir("/Users/erjo3868/repos/hypedsearch/hypedsearch")
print(f"cwd = {os.getcwd()}")

cwd = /Users/erjo3868/repos/hypedsearch/hypedsearch


# Create protein-product ion DBs

In [7]:
import pandas as pd
from src.erik import load_comet_data
from src.erik_constants import PRODUCT_ION_TABLE
from src.fasta_utils import get_proteins_from_fasta, get_specific_protein_from_fasta
from src.lookups.protein_product_ion_db import create_protein_product_ion_db, get_average_mass_search_time
from experiments.ions_per_mass import main

fasta_path = "fastas/Uniprot_mouse.fasta"
proteins = list(get_proteins_from_fasta(fasta_path=fasta_path))
proteins = proteins[:10]
# db = create_protein_product_ion_db(
#     protein_seqs=protein_seqs, 
#     db_path=":memory:",
#     max_kmer_len=30,
#     charges_to_consider=[1, 2, 3]
# )

# protein_name = "sp|P99027|RLA2_MOUSE"
# protein = get_specific_protein_from_fasta(fasta_path=fasta_path, protein_name=protein_name)

db_path = "dbs/test.proteinproduction.db"
db = create_protein_product_ion_db(
    db_path=db_path,
    protein_seqs=[protein.sequence for protein in proteins[:10]],
    charges_to_consider=[1, 2],
    max_kmer_len=30,
)

df = main(db_path=db_path, sample_size=10, ppm_tolerance=10, output_dir="dbs")
df
output_path = "dbs/test.proteinproduction.db_sampleSize=10_ppmTol=10.h5"
pd.read_hdf(output_path, key="df") 

,mass,time,num_matching_ions,len_search_ion,num_matching_ions_by_charge
0,418.231636,0.015255,29,7,"{1: 20, 2: 9}"
1,1642.902890,0.014413,31,28,"{1: 19, 2: 12}"
2,1363.740142,0.014352,27,23,"{2: 10, 1: 17}"
3,499.772522,0.015106,13,12,{2: 13}
4,723.901191,0.014697,12,13,{2: 12}
5,982.994202,0.014510,22,18,{2: 22}
6,2526.160782,0.013677,5,23,{1: 5}
7,1044.544590,0.014750,32,17,"{1: 15, 2: 17}"
8,1164.113703,0.014985,17,20,{2: 17}
9,504.225831,0.015100,23,8,"{1: 13, 2: 10}"


,mass,time,num_matching_ions,len_search_ion,num_matching_ions_by_charge
0,418.231636,0.015255,29,7,"{1: 20, 2: 9}"
1,1642.902890,0.014413,31,28,"{1: 19, 2: 12}"
2,1363.740142,0.014352,27,23,"{2: 10, 1: 17}"
3,499.772522,0.015106,13,12,{2: 13}
4,723.901191,0.014697,12,13,{2: 12}
5,982.994202,0.014510,22,18,{2: 22}
6,2526.160782,0.013677,5,23,{1: 5}
7,1044.544590,0.014750,32,17,"{1: 15, 2: 17}"
8,1164.113703,0.014985,17,20,{2: 17}
9,504.225831,0.015100,23,8,"{1: 13, 2: 10}"


In [36]:
from collections import Counter


type(Counter([1,2,1,2,3,4]))

collections.Counter

In [ ]:
proteins.sequence

[Protein(sequence='MGRLTSSFLLLIVPAYVLSQVTLKESGPGILQPSQTLSLTCSFSGFSLSTSNMGIGWIRQPSGKGLEWLAHIWWNDDKYYNPSLKSRLTISKDTSNNQVFLKITSVDTADTATYYCAQI', protein_id=0, desc='tr|A0A0A6YY60|A0A0A6YY60_MOUSE Immunoglobulin heavy variable V8-5 (Fragment) OS=Mus musculus OX=10090 GN=Ighv8-5 PE=1 SV=1'),
 Protein(sequence='MGESNGEAFLAFKTSASPTAPVTTNPMDETLPGSINIRILIPKLMIIIFGLVGLMGNAIVFWLLGFHLRRNAFSVYILNLALADFLFLLSSIIASTLFLLKVSYLSIIFHLCFNTIIMVVYITGISMLSAISTECCLSVLCPTWYRCHRPVHTSTVMCAVIWVLSLLICILNSYFCAVLHTRYDNDNECLATNIFTASYMIFLLVVLCLSSLALLARLFCGAGQMKLTRFHVTILLTLLVFLLCGLPFVIYCILLFKIKDDFHVLDVNLYLALEVLTAINSCANPIIYFFVGSFRHRLKHQTLKMVLQSALQDTPETAENMVEMSSNKAEP', protein_id=1, desc='tr|A0A0R4J1Z0|A0A0R4J1Z0_MOUSE MAS-related GPR, member A2A OS=Mus musculus OX=10090 GN=Mrgpra2a PE=4 SV=1'),
 Protein(sequence='MYFTFNCLFLLLQSLFTPVEMTRKSCYEFWNPSTYMKADIILGAFIPIFILLEGPKEIQQFIWIQPDIVLENILLQWKNYQYLMTLYFAIDEINKDTHLLPNVTLGFHIYNAFNFHPRTLEGPLMWLSGRNEFIPNYKCKTQYKALGIISGTRSEYSAGIGSLLERYKIPQVSFALLLQVNYGPFDSMLSDKDTFPSLYQMSPKDTALTQGV

# How long does it take to query the protein-product ion DB by mass? 

In [ ]:
from src.lookups.protein_product_ion_db import load_existing_protein_product_ion_db

db_path = 
db = load_existing_protein_product_ion_db(db_path=db_path)
query = f"SELECT {MASS} FROM {PRODUCT_ION_TABLE} ORDER BY RANDOM() LIMIT {sample_size};"
db.run_query(query=query)